# Quality assurance and quality control

## Low level winds and temperatures

In [2]:
%%capture
import glob
import os
import sys

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import numpy as np
import pandas as pd
import xarray as xr
from metpy.units import units

sys.path.insert(1, "/Users/lukeconibear/repos/wxapps/")

from apps.low_level_winds.src.low_level_winds import low_level_winds, utils

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.sans-serif": ["Computer Modern Roman"],
        "axes.grid": False,
        "savefig.dpi": 700,
        "figure.figsize": [8, 6],
        "font.size": 14,
    }
)
plt.style.use(["seaborn-colorblind"])
xr.set_options(display_expand_data=False)

In [3]:
batch_timestamps = pd.date_range(start="2022-05-02", end="2022-06-30", freq="6H")
filename_variable = "wind_temp"
data_variables = [
    "air_temperature_low_level",
    "wind_speed_low_level",
    "wind_direction_low_level",
]

In [25]:
hrrr_raw_filenames = sorted(glob.glob('../../wxapps/apps/low_level_winds/tests/test_data/HRRR*.grb2'))
hrrr_out_filenames = sorted(glob.glob('../../pangeo_tools/low_level_winds_temps/processed_data/*hrrr*.nc'))

In [34]:
def run_half_hrrr_conus(hrrr_filename):
    ds_hrrr_hybrid = low_level_winds.open_hrrr_dataset(
        filename=hrrr_filename,
        filter_by_keys={"typeOfLevel": "hybrid"},
        data_vars=["gh", "t", "u", "v"],
    )

    levels_to_keep = [
        lev
        for lev in range(len(ds_hrrr_hybrid.hybrid))
        if ds_hrrr_hybrid["gh"].isel(hybrid=lev).min().values <= 300
    ]
    ds_hrrr_hybrid = ds_hrrr_hybrid.isel(hybrid=levels_to_keep)
    ds_hrrr_surface = low_level_winds.open_hrrr_dataset(
        filename=hrrr_filename,
        filter_by_keys={"typeOfLevel": "surface", "stepType": "instant"},
        data_vars=["orog"],
    )

    resolution_lat = 0.027  # approx 3km
    resolution_lon = 0.027  # approx 3km
    max_lat = np.round(ds_hrrr_hybrid.lat.max().values, 0)
    min_lat = np.round(ds_hrrr_hybrid.lat.min().values, 0)
    max_lon = (
        np.round(ds_hrrr_hybrid.lon.max().values, 0) - 360
    )  # needs to be between -180 and 180
    min_lon = (
        np.round(ds_hrrr_hybrid.lon.min().values, 0) - 360
    )  # needs to be between -180 and 180
    gridcells_lat_old = len(ds_hrrr_hybrid.y.values)
    gridcells_lon_old = len(ds_hrrr_hybrid.x.values)
    gridcells_lat_new = int((max_lat - min_lat) / resolution_lat)
    gridcells_lon_new = int((max_lon - min_lon) / resolution_lon)
    reuse_weights = True
    regridding_weights_filename = f"bilinear_{gridcells_lat_old}x{gridcells_lon_old}_{gridcells_lat_new}x{gridcells_lon_new}.nc"
    periodic = False
    save_weights = True

    ds_hrrr_hybrid_regridded = low_level_winds.regrid_ds(
        ds_hrrr_hybrid,
        resolution_lat,
        resolution_lon,
        max_lat,
        min_lat,
        max_lon,
        min_lon,
        reuse_weights,
        regridding_weights_filename,
        periodic,
        save_weights,
    )

    ds_hrrr_surface_regridded = low_level_winds.regrid_ds(
        ds_hrrr_surface,
        resolution_lat,
        resolution_lon,
        max_lat,
        min_lat,
        max_lon,
        min_lon,
        reuse_weights,
        regridding_weights_filename,
        periodic,
        save_weights,

    )

    size_y = len(ds_hrrr_hybrid_regridded.lat)
    size_x = len(ds_hrrr_hybrid_regridded.lon)
    lats = ds_hrrr_hybrid_regridded.lat
    lons = ds_hrrr_hybrid_regridded.lon
    heights_new = np.array([10, 20, 30, 40, 50, 80, 100, 150, 200, 250, 300])
    da_hrrr_heights_new = low_level_winds.create_array_for_heights(
        size_y,
        size_x,
        heights_new,
        lons,
        lats,
    )

    da_hrrr_heights_above_surface_old = (
        ds_hrrr_hybrid_regridded["gh"] - ds_hrrr_surface_regridded["orog"]
    )

    # change 0.0 values (added from regridding) to np.nan
    da_hrrr_heights_above_surface_old = xr.where(
        da_hrrr_heights_above_surface_old == 0,
        np.nan,
        da_hrrr_heights_above_surface_old,
    )

    da_hrrr_air_temperature_regridded = xr.where(
        ds_hrrr_hybrid_regridded["t"] == 0,
        np.nan,
        ds_hrrr_hybrid_regridded["t"],
    )

    da_hrrr_air_temperature_regridded = (
        (da_hrrr_air_temperature_regridded * units("K"))
        .metpy.convert_units("degC")
        .metpy.dequantify()
    )

    da_hrrr_u_wind_component_regridded = ds_hrrr_hybrid_regridded["u"] * units("m/s")
    da_hrrr_v_wind_component_regridded = ds_hrrr_hybrid_regridded["v"] * units("m/s")

    da_hrrr_wind_speed_regridded = mpcalc.wind_speed(
        da_hrrr_u_wind_component_regridded, da_hrrr_v_wind_component_regridded
    ).metpy.dequantify()
    da_hrrr_wind_direction_regridded = mpcalc.wind_direction(
        da_hrrr_u_wind_component_regridded, da_hrrr_v_wind_component_regridded
    ).metpy.dequantify()

    # change 0.0 values (added from regridding) to np.nan
    da_hrrr_wind_speed_regridded = xr.where(
        da_hrrr_wind_speed_regridded == 0, np.nan, da_hrrr_wind_speed_regridded
    )
    da_hrrr_wind_direction_regridded = xr.where(
        da_hrrr_wind_direction_regridded == 0,
        np.nan,
        da_hrrr_wind_direction_regridded,
    )

    ds_hrrr_regridded_combined = xr.combine_by_coords(
        [
            xr.Dataset(
                {"air_temperature_low_level": ds_hrrr_air_temperature_regridded["t"]}
            ),
            xr.Dataset({"wind_speed_low_level": da_hrrr_wind_speed_regridded}),
            xr.Dataset({"wind_direction_low_level": da_hrrr_wind_direction_regridded}),
        ]
    )
    return ds_hrrr_regridded_combined

In [ ]:
for index in range(len(hrrr_filenames)):
    hrrr_raw_filename = hrrr_raw_filenames[index]
    hrrr_out_filename = hrrr_out_filenames[index]
    
    ds_hrrr_regridded_combined = run_half_hrrr_conus(hrrr_filename)
    ds_hrrr_out = xr.open_dataset(hrrr_out_filename)

In [32]:
hrrr_raw_filename = hrrr_raw_filenames[index]
hrrr_out_filename = hrrr_out_filenames[index]

ds_hrrr_regridded_combined = run_half_hrrr_conus(hrrr_filename)
ds_hrrr_out = xr.open_dataset(hrrr_out_filename)

NameError: name 'open_hrrr_dataset' is not defined

In [26]:
ds_hrrr_regridded_combined

['../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071421_f000.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071421_f001.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071421_f002.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071422_f000.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071422_f001.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071422_f002.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071423_f000.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071423_f001.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071423_f002.grb2',
 '../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071500_f000.grb2']

In [33]:
ds_hrrr_out

NameError: name 'ds_hrrr_out' is not defined

In [11]:
!ls ../../pangeo_tools/low_level_winds_temps/processed_data/*hrrr*.nc

../../pangeo_tools/low_level_winds_temps/processed_data/hrrr.t00z.wrfnatf00_nan.nc


In [ ]:
"low_level_winds_[hrrr][conus][wind_temp]_20220714T210000Z_[20220714T220000Z].nc"

In [13]:
hrrr_filename = "../../wxapps/apps/low_level_winds/tests/test_data/HRRR_NAT_2022071421_f000.grb2"

In [14]:
ds_hrrr_hybrid = low_level_winds.open_hrrr_dataset(
    filename=hrrr_filename,
    filter_by_keys={"typeOfLevel": "hybrid"},
    data_vars=["gh", "t", "u", "v"],
)

In [15]:
ds_hrrr_hybrid

<xarray.Dataset>
Dimensions:     (hybrid: 50, y: 1059, x: 1799)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 47.0 48.0 49.0 50.0
    lat         (y, x) float64 ...
    lon         (y, x) float64 ...
    valid_time  datetime64[ns] ...
    metpy_crs   object Projection: lambert_conformal_conic
Dimensions without coordinates: y, x
Data variables:
    gh          (hybrid, y, x) float32 ...
    t           (hybrid, y, x) float32 ...
    u           (hybrid, y, x) float32 ...
    v           (hybrid, y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2022-07-15T13:46 GRIB to CDM+CF via cfgrib-0.9.1...